In [ ]:
# Komórka 1: Instalacja i importowanie bibliotek
# W środowisku Jupyter może być konieczne zainstalowanie pycaret
# !pip install python-dotenv boto3 pandas scikit-learn joblib "pycaret[full]"
from dotenv import load_dotenv
import pandas as pd
import boto3
from botocore.client import Config
import os
import joblib
import io
# Importujemy moduł regresji z PyCaret
from pycaret.regression import setup, compare_models, finalize_model, save_model as pycaret_save_model

# Komórka 2: Wczytanie zmiennych środowiskowych
load_dotenv()

# Pobieranie konfiguracji ze zmiennych środowiskowych
access_key = os.getenv('DO_SPACES_KEY')
secret_key = os.getenv('DO_SPACES_SECRET')
endpoint_url = os.getenv('DO_SPACES_ENDPOINT_URL')
bucket_name = os.getenv('DO_SPACES_BUCKET')


# Komórka 3: Konfiguracja klienta S3 (DigitalOcean Spaces)
# --- POPRAWKA ---
# Bardziej niezawodny sposób wyodrębniania kodu regionu z adresu URL.
# Np. z "https://fra1.digitaloceanspaces.com" wyciągnie "fra1".
try:
    region_name = endpoint_url.split('//')[1].split('.')[0]
except (IndexError, AttributeError):
    print("BŁĄD: Nieprawidłowy format DO_SPACES_ENDPOINT_URL. Sprawdź plik .env.")
    region_name = None

session = boto3.session.Session()
client = session.client('s3',
                        config=Config(s3={'addressing_style': 'virtual'}),
                        region_name=region_name,
                        endpoint_url=endpoint_url,
                        aws_access_key_id=access_key,
                        aws_secret_access_key=secret_key)

print(f"Połączono z DigitalOcean Spaces w regionie: {region_name}")

# Komórka 4: Funkcja do wczytywania danych ze Spaces
def load_data_from_spaces(file_keys):
    """Wczytuje wiele plików CSV ze Spaces i łączy je w jeden DataFrame."""
    all_dfs = []
    for key in file_keys:
        try:
            response = client.get_object(Bucket=bucket_name, Key=key)
            csv_content = response['Body'].read()
            df = pd.read_csv(io.BytesIO(csv_content), sep=';')
            all_dfs.append(df)
            print(f"Pomyślnie wczytano plik: {key}")
        except Exception as e:
            print(f"Błąd podczas wczytywania pliku {key}: {e}")
            return None
    if not all_dfs:
        return pd.DataFrame()
    return pd.concat(all_dfs, ignore_index=True)

# Komórka 5: Funkcja do konwersji czasu na sekundy
def time_to_seconds(time_str):
    """Konwertuje czas w formacie HH:MM:SS lub MM:SS na sekundy."""
    if pd.isna(time_str):
        return None
    parts = str(time_str).split(':')
    if len(parts) == 3:
        h, m, s = map(int, parts)
        return h * 3600 + m * 60 + s
    elif len(parts) == 2:
        m, s = map(int, parts)
        return m * 60 + s
    return None

# Komórka 6: Czyszczenie i przygotowanie danych
def clean_and_prepare_data(df):
    """Czyści i przygotowuje dane do trenowania modelu."""
    print("Rozpoczynam czyszczenie danych...")
    cols_to_use = ['plec', 'wiek', 'czas_5km', 'czas_netto']
    df_clean = df[cols_to_use].copy()
    df_clean['czas_netto_s'] = df_clean['czas_netto'].apply(time_to_seconds)
    df_clean['czas_5km_s'] = df_clean['czas_5km'].apply(time_to_seconds)
    df_clean.dropna(subset=['czas_netto_s', 'czas_5km_s', 'wiek', 'plec'], inplace=True)
    df_clean['tempo_5km_s_na_km'] = df_clean['czas_5km_s'] / 5
    df_clean.drop(['czas_netto', 'czas_5km', 'czas_5km_s'], axis=1, inplace=True)
    df_clean['wiek'] = df_clean['wiek'].astype(int)
    print(f"Czyszczenie zakończone. Pozostało {len(df_clean)} wierszy.")
    print("Przykładowe dane po czyszczeniu:")
    print(df_clean.head())
    return df_clean

# Komórka 7: Trenowanie i wybór najlepszego modelu za pomocą PyCaret
def train_and_select_model(df):
    """Używa PyCaret do automatycznego znalezienia i wytrenowania najlepszego modelu."""
    print("Rozpoczynam proces AutoML z PyCaret...")
    
    s = setup(data=df, 
              target='czas_netto_s', 
              categorical_features=['plec'],
              numeric_features=['wiek', 'tempo_5km_s_na_km'],
              session_id=123,
              verbose=False,
              log_experiment=False
             )
    
    print("Porównuję modele, aby znaleźć najlepszy...")
    best_model = compare_models(sort='MAE')
    
    print("\nWyniki porównania modeli:")
    print(best_model)
    
    final_model = finalize_model(best_model)
    
    print("\nSfinalizowany najlepszy model:")
    print(final_model)
    
    return final_model

# Komórka 8: Funkcja do zapisywania modelu
def save_and_upload_model(model, local_path, remote_path):
    """Zapisuje model lokalnie i wysyła do DigitalOcean Spaces."""
    pycaret_save_model(model, local_path)
    print(f"Model zapisany lokalnie w pliku: {local_path}.pkl")
    
    try:
        with open(f"{local_path}.pkl", 'rb') as f:
            client.upload_fileobj(f, bucket_name, remote_path)
        print(f"Model pomyślnie wysłany do DigitalOcean Spaces jako: {remote_path}")
    except Exception as e:
        print(f"Błąd podczas wysyłania modelu do Spaces: {e}")

# Komórka 9: Główna logika pipeline'u
if __name__ == '__main__':
    files = ['halfmarathon_wroclaw_2023__final.csv', 'halfmarathon_wroclaw_2024__final.csv']
    raw_data = load_data_from_spaces(files)
    
    if raw_data is not None and not raw_data.empty:
        cleaned_data = clean_and_prepare_data(raw_data)
        
        best_pipeline = train_and_select_model(cleaned_data)
        
        local_model_path_no_ext = 'halfmarathon_pipeline'
        remote_model_path = 'models/halfmarathon_pipeline.pkl'
        
        save_and_upload_model(best_pipeline, local_model_path_no_ext, remote_model_path)
        
        print("\nPipeline AutoML zakończony sukcesem!")
